In [ ]:
# secrets.env should contain OPENAI_API_KEY
# !pip install openai python-dotenv

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [12]:
load_dotenv(dotenv_path='../secrets.env')
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [22]:
def get_stargazing_status(lat, long):
    return {
        "stargazing": 8
    }

functions = [
    {
        "name": "get_stargazing_status",
        "description": "Get the stargazing quality of a location, given its latitude and longitude.",
        "parameters": {
            "type": "object",
            "properties": {
                "lat": {
                    "type": "number",
                    "description": "The latitude of the location to check."
                },
                "long": {
                    "type": "number",
                    "description": "The longitude of the location to check."
                }
            },
            "required": ["lat", "long"]
        }
    }
]

In [23]:
response = client.chat.completions.create(
    model="gpt-4",  # Changed from "gpt-4o" to "gpt-4"
    messages=[
        {"role": "user", "content": "What's the stargazing like at latitude 45, longitude 55? Use the function call to get the stargazing status from our database."}
    ],
    functions=functions,
    function_call="auto"
)

if response.choices[0].message.function_call:  # Changed from .function_call to .message.function_call
    function_call = response.choices[0].message.function_call
    function_name = function_call.name
    function_args = eval(function_call.arguments)
    
    # Call the function
    function_response = get_stargazing_status(**function_args)  # Changed from get_current_weather to get_status
    
    # Send the function result back to the model
    second_response = client.chat.completions.create(
        model="gpt-4",  # Changed from "gpt-4o" to "gpt-4"
        messages=[
            {"role": "user", "content": "What's the stargazing like at latitude 45, longitude 55?"},
            {"role": "function", "name": function_name, "content": str(function_response)}
        ]
    )
    
    print(second_response.choices[0].message.content)
else:
    print(response.choices[0].message.content)

The stargazing at latitude 45, longitude 55 is fairly good, ranking at an 8 out of 10. This suggests clear skies with good visibility of stars and other celestial bodies. As always, stargazing conditions can vary due to weather and light pollution.


# Simple OpenAI Function Calling Example

def get_current_weather(location, unit="celsius"):
    """Get the current weather in a given location"""
    # Dummy function to simulate getting weather data
    return {
        "location": location,
        "temperature": 22,
        "unit": unit,
        "forecast": ["sunny", "windy"]
    }

# Define the function for the API to use
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What's the weather like in Boston?"}
    ],
    functions=functions,
    function_call="auto"
)

# Check if the model wants to call a function
if response.choices[0].function_call:
    function_call = response.choices[0].function_call
    function_name = function_call.name
    function_args = eval(function_call.arguments)
    
    # Call the function
    function_response = get_current_weather(**function_args)
    
    # Send the function result back to the model
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "user", "content": "What's the weather like in Boston?"},
            {"role": "function", "name": function_name, "content": str(function_response)}
        ]
    )
    
    print(second_response.choices[0].message.content)
else:
    print(response.choices[0].message.content)
